In [ ]:
import os
import cv2
import numpy as np
from collections import deque
from tensorflow.keras.models import load_model
import yt_dlp

# -----------------------
# CONFIGURATIONS
# -----------------------
SEQUENCE_LENGTH = 20   
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
MODEL_PATH = "convLSTM_date_time_2025_08_20_23_53_37_loss_0.2337915450334549_acc_0.9180035591125488.h5"

# Load trained model
model = load_model(MODEL_PATH)

# Action classes (update with your dataset labels)
CLASSES_LIST = ["Normal", "Violence"]

# Queue to store last N frames
frames_queue = deque(maxlen=SEQUENCE_LENGTH)


In [24]:
def youtube_video_download(output_dir, video_url):
    ydl_opts = {
        'outtmpl': f'{output_dir}/%(title)s.%(ext)s',
        'format': 'mp4[height<=720]+bestaudio/best[ext=mp4]',  
        'merge_output_format': 'mp4',
        'prefer_ffmpeg': False, 
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=True)
        return info.get('title', None)
    
def predict_video(video_path,output_file_path,seq_len):
    video_reader = cv2.VideoCapture(video_path)
    org_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    org_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M','P','4','V'), video_reader.get(cv2.CAP_PROP_FPS), (org_video_width, org_video_height))
    frames_queue = deque(maxlen=seq_len)
    predicted_class_name = ""
    while True:
        ok, frame = video_reader.read()
        if not ok:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)
        if len(frames_queue) == seq_len:
            predicted_label_prob = model.predict(np.expand_dims(frames_queue, axis=0))[0]
            predicted_label = np.argmax(predicted_label_prob)
            predicted_class_name = CLASSES_LIST[predicted_label]
        cv2.putText(frame, f"Predicted: {predicted_class_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        video_writer.write(frame)
    video_reader.release()
    video_writer.release()

def predict_camera():
    video_reader = cv2.VideoCapture(0)
    org_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    org_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    frames_queue = deque(maxlen=SEQUENCE_LENGTH)
    predicted_class_name = ""
    predicted_label = 0.0
    while True:
        ok, frame = video_reader.read()
        if not ok:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)
        if len(frames_queue) == SEQUENCE_LENGTH:
            predicted_label_prob = model.predict(np.expand_dims(frames_queue, axis=0))[0]
            predicted_label = np.argmax(predicted_label_prob)
            predicted_class_name = CLASSES_LIST[predicted_label]
        cv2.putText(frame, f"Predicted: {predicted_class_name} ({predicted_label:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow("LRCN Action Recognition (Live)", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    video_reader.release()
    cv2.destroyAllWindows


In [25]:
# os.makedirs('downloaded_videos', exist_ok=True)
# video_title = youtube_video_download('downloaded_videos', 'https://youtube.com/shorts/mymElEe0iMg?si=vFOzkJKm6uOrnIVk')
# video_path = f'downloaded_videos/{video_title}.mp4'

# os.makedirs('output_videos', exist_ok=True)
# output_file_path = f'output_videos/predicted_{video_title}_seqLen{SEQUENCE_LENGTH}.mp4'
# predict_video(video_path, output_file_path,SEQUENCE_LENGTH)

predict_camera()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━